In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AffinityPropagation
from collections import Counter
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# from textblob.sentiments import NaiveBayesAnalyzer
import re
import pandas as pd
import sqlite3
# from textblob import TextBlob

import seaborn as sns
sns.set(color_codes=True)

# encoding=utf8
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

In [ ]:
cd Desktop/

In [ ]:
df = pd.read_csv('../pitchfork3.csv')

In [20]:
df = pd.read_csv('../pitchfork3.csv')

years = np.arange(1999, 2018)
df_new = df[df['new_album'] == 1]
df_reissue = df[df['new_album'] == 0]

df_spotify = pd.read_csv('spotify_with_rank.csv')

df_merged = df_new.merge(df_spotify, how='left', on=['artist', 'title'])

In [21]:
df_sample = df_merged[df_merged['best_new_music'] == 1]

In [22]:
# 4. Limit the number of features and see if the words of the topics change.
vectorizer = TfidfVectorizer(stop_words='english', max_features=500, max_df=.4)
X = vectorizer.fit_transform(df_sample['content'].values.astype('U'))
features = vectorizer.get_feature_names()
kmeans = KMeans(n_clusters=11)
kmeans.fit(X)
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
print "top features for each cluster with 500 max features:"
for num, centroid in enumerate(top_centroids):
    print "%d: %s" % (num, ", ".join(features[i] for i in centroid))

top features for each cluster with 500 max features:
0: punk, post, art, energy, live, drum, bands, hooks, right, let
1: cox, dream, solo, loops, ambient, bass, weird, gets, indie, era
2: kanye, west, rapper, drake, chance, rap, mixtape, soul, future, hip
3: rk, bj, folk, broken, songwriter, piano, singer, chords, moment, digital
4: indie, girls, bands, ll, heart, lo, chorus, fi, want, title
5: black, metal, noise, guitars, blood, minutes, group, power, pink, live
6: rap, hop, hip, beats, rapper, jay, mike, beat, mixtape, production
7: techno, electronic, dance, field, producer, bass, piano, noise, space, ambient
8: dance, disco, house, electro, electronic, vocal, duo, dj, vocals, bass
9: death, sings, know, woman, end, heart, real, old, things, youth
10: vocal, kind, bear, records, collective, folk, animal, vocals, dream, jazz


In [23]:
df_sample['tf-idf'] = list(X.toarray())

/Users/andrewbashford/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# 5. Print out the titles of a random sample of the articles assigned to each
# cluster to get a sense of the topic.
assigned_cluster = kmeans.transform(X).argmin(axis=1)
for i in range(kmeans.n_clusters):
    cluster = np.arange(0, X.shape[0])[assigned_cluster==i]
    sample_articles = np.random.choice(cluster, 8, replace=False)
    print "cluster %d:" % i
    for article in sample_articles:
        print "    %s" % df.iloc[article][5]

In [ ]:
df_merged['cluster']  =  kmeans.predict(X)

In [7]:
import graphlab

In [169]:
df_sample.columns

Index([u'Unnamed: 0_x', u'Unnamed: 0.1', u'index', u'reviewid', u'title',
       u'artist', u'url', u'score', u'best_new_music', u'best_new_reissue',
       u'author', u'pub_date', u'pub_weekday', u'pub_day', u'pub_month',
       u'pub_year', u'year', u'genre_electronic', u'genre_experimental',
       u'genre_folk/country', u'genre_global', u'genre_jazz', u'genre_metal',
       u'genre_pop/r&b', u'genre_rap', u'genre_rock', u'content', u'abstract',
       u'reissue', u'num_years_since_release', u'new_album', u'abstract_adj',
       u'content_adj', u'cont_polarity', u'cont_subjectivity', u'abs_polarity',
       u'abs_subjectivity', u'abs_adj_polarity', u'abs_adj_subjectivity',
       u'cont_adj_polarity', u'cont_adj_subjectivity', u'word_count',
       u'adj_count', u'adj_freq', u'score_bin', u'avg_word_length',
       u'adj_word_length', u'num_prev_reviews', u'avg_score_of_prev_reviews',
       u'number_prev_bnm', u'era', u'Unnamed: 0_y', u'album_name',
       u'album_release_year', u'

In [182]:
df_sample['danceability rounded'] = df_sample['danceability mean'].round(2) * 100 // 10

/Users/andrewbashford/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [187]:
sf = graphlab.SFrame(df_sample[['reviewid', 'tf-idf', 'score_bin']])

In [188]:
recommender = graphlab.recommender.item_content_recommender.create(sf, item_id='reviewid')

('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['tf-idf', 'score_bin']
Excluded Features : ['reviewid']

Column     Type   Interpretation  Transforms  Output Type
---------  -----  --------------  ----------  -----------
tf-idf     array  vector          None        array      
score_bin  int    numerical       None        int        

)


Recsys training: model = item_content_recommender

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

Starting pairwise querying.

...

Data has 0 observations with 0 users and 644 items.

Data prepared in: 0.087639s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.006198s

In [111]:
pd.options.display.max_colwidth = 100

In [ ]:
recommender.get_similar_items(['reviewid'])

In [66]:
top_5_similar = recommender.get_similar_items(['5929e803c0084474cd0c5102'], k=5)['similar']

In [61]:
recommender.get_similar_items(['https://pitchfork.com/reviews/albums/18172-kanye-west-yeezus/'])

reviewid,similar,score,rank


In [189]:
def get_similar(df, url, recommender):
    reviewid = df[df['url'] == url].iloc[0][3]
    print reviewid
    top_5_similar = recommender.get_similar_items([reviewid], k=10)['similar']
    print top_5_similar
    return df[df['reviewid'].isin(top_5_similar)][['artist','title', 'score', 'url']]

In [167]:
df_merged[df_merged['url'] == 'https://pitchfork.com/reviews/albums/18172-kanye-west-yeezus/'].iloc[0][3]

'5929e5fd5e6ef959693253fc'

In [190]:
get_similar(df_merged, 'https://pitchfork.com/reviews/albums/452-funeral/', recommender)

5929e2ad5e6ef95969325178
['5929d8995e6ef95969324701', '5929dbf013d197565213cf22', '5929e3f8eb335119a49ef19e', '5929d8d1eb335119a49ee663', '5929e052eb335119a49eedac', '5929e56bd7a71d1ea569a70e', '5929e4e3c0084474cd0c4fc2', '5929d8305e6ef9596932469f', '5929e7425e6ef95969325475', '5929dc970c2bba1b7de02546']


,artist,title,score,url
762,mount eerie,a crow looked at me,9.0,https://pitchfork.com/reviews/albums/22970-a-crow-looked-at-me/
1694,anohni,hopelessness,9.0,https://pitchfork.com/reviews/albums/21819-hopelessness/
3986,sun kil moon,benji,9.2,https://pitchfork.com/reviews/albums/18860-sun-kil-moon-benji/
4959,my bloody valentine,m b v,9.1,https://pitchfork.com/reviews/albums/17726-mbv/
5212,kendrick lamar,"good kid, m.a.a.d city",9.5,https://pitchfork.com/reviews/albums/17253-good-kid-maad-city/
6529,bon iver,"bon iver, bon iver",9.5,https://pitchfork.com/reviews/albums/15551-bon-iver/
9394,tv on the radio,dear science,9.2,https://pitchfork.com/reviews/albums/12246-dear-science/
12751,sufjan stevens,illinois,9.2,https://pitchfork.com/reviews/albums/7514-illinois/
14542,the rapture,echoes,9.0,https://pitchfork.com/reviews/albums/6693-echoes/
14935,sunburned hand of the man,headdress,9.0,https://pitchfork.com/reviews/albums/7569-headdress/


In [98]:
recommender.get_similar_items(['5929e5fd5e6ef959693253fc'])['similar']

dtype: str
Rows: 10
['5929dfdb13d197565213d322', '5929dc970c2bba1b7de02546', '5929d930d7a71d1ea5699a8a', '5929d8305e6ef9596932469f', '5929e544c0084474cd0c500b', '5929dc95eb335119a49ee9f3', '5929e0cc5e6ef95969324f96', '5929e40ad7a71d1ea569a5e8', '5929e332c0084474cd0c4e48', '5929d83513d197565213cb98']

In [75]:
reviewid = df[df['url'] == 'https://pitchfork.com/reviews/albums/18172-kanye-west-yeezus/']['reviewid']

In [97]:
df[df['url'] == 'https://pitchfork.com/reviews/albums/18172-kanye-west-yeezus/'].iloc[0][3]

'5929e5fd5e6ef959693253fc'

In [49]:
df[df['reviewid'] == '5929e391c0084474cd0c4eb1'][['artist','title']]

,artist,title
10454,cymbals eat guitars,why there are mountains
